# Network Analysis of the Chromatin

In [1]:
# ================================
#           libraries
# ================================

# %pip install -q MDTraj    # leave this uncommented if running it on Google Colab
import mdtraj as md

import pandas as pd
import numpy as np
import scipy.ndimage as nd
import math
from tqdm import tqdm
from random import random
import plotly.graph_objects as go
import plotly.io as pio

import urllib.request
import ssl
from copy import deepcopy


In [4]:
url = 'https://calla.rnet.missouri.edu/genome3d/GSDB/Database/AX9716PF/GSE105544_ENCFF010WBP/VC/LorDG/chr1.pdb'

In [5]:
# Create unverified context for this specific download
ssl_context = ssl._create_unverified_context()

# Download the file
with urllib.request.urlopen(url, context=ssl_context) as response:
    with open('chr1.pdb', 'wb') as out_file:
        out_file.write(response.read())

# Load from local file
chr_str = md.load_pdb('chr1.pdb')
coords = chr_str.xyz.reshape(-1, 3)

### Distance From Center of Mass

In [6]:
com = md.compute_center_of_mass(chr_str)

chr_com = []
for coord in coords:
    distances = np.linalg.norm(com - coord, axis=1)
    min_dist = np.min(distances)
    chr_com.append ([*coord, min_dist])

chr_com = np.array(chr_com)

chr_com.shape


(451, 4)

### Rolling Mean

In [13]:
window_size = 50
jump = 1

smooth = []
for i in range(0, chr_com.shape[0] - window_size, jump):
    x = np.mean(chr_com[i:i + window_size, 0])
    y = np.mean(chr_com[i:i + window_size, 1])
    z = np.mean(chr_com[i:i + window_size, 2])
    smooth.append([x, y, z])

smooth = np.array(smooth)

In [ ]:
line_1_dict = dict(width=5, color='skyblue')
line_2_dict = dict(width=10, color='red')
title_dict = dict(text='Rolling Mean', x=0.5, font=dict(size=24))
data = go.Scatter3d(
    x=smooth[:, 0], 
    y=smooth[:, 1], 
    z=smooth[:, 2], 
    mode='lines', 
    line=line_2_dict, 
    name='rolling_mean'
)
fig = go.Figure(data)
fig.add_trace(go.Scatter3d( 
                    x=coords[:, 0], 
                    y=coords[:, 1], 
                    z=coords[:, 2], 
                    mode='lines', 
                    line=line_1_dict, 
                    name='chromosome',
                    opacity=0.5
))
fig.update_layout(
    height=600, 
    width=600, 
    title=title_dict,
    template='plotly_dark'  # Move template here
)
fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
    ),
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    paper_bgcolor='rgba(0,0,0,0)',
    showlegend=False
)
fig.show()


### Distance From Smooth Line

In [44]:
# smooth # chr_com

chr_com_smooth = []
for coord in chr_com:
    distances = np.linalg.norm(coord[:3] - smooth, axis=1)
    min_dist = np.min(distances)

    chr_com_smooth.append([*coord, min_dist])

chr_com_smooth = np.array(chr_com_smooth)

chr_com_smooth.shape

(451, 5)

In [45]:
# ==========================================================
#       x --- y --- z --- dist_1p_com --- dist_rm_com
# ==========================================================

chr_com_smooth[1:5, :]

array([[-0.3026    ,  0.43669999, -0.26809999,  0.59653344,  0.20053388],
       [-0.21709999,  0.50260001, -0.26159999,  0.60691177,  0.11906397],
       [-0.0999    ,  0.60250002, -0.21600001,  0.64595065,  0.12114562],
       [ 0.017     ,  0.65240002, -0.1841    ,  0.6748825 ,  0.22334391]])